# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
file_path = "../output_data/cities.csv"
cities_df = pd.read_csv(file_path)
cities_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Convert Humidity to float and store
# HINT: be sure to handle NaN values
city_df_complete = cities_df.dropna()
#humidity_df = city_df_complete.Humidity.astype(float)
city_df_complete

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,ostrovnoy,2,RU,1558378754,72,68.05,39.51,37.50,7.16
1,1,mahebourg,75,MU,1558378503,74,-20.41,57.70,78.80,11.41
2,2,qaanaaq,25,GL,1558378755,73,77.48,-69.36,22.20,2.37
3,3,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
4,4,cape town,20,ZA,1558378755,76,-33.93,18.42,55.99,8.05
...,...,...,...,...,...,...,...,...,...,...
543,543,yabrud,0,SY,1558378840,32,33.97,36.66,80.60,8.05
544,544,paraiso,5,MX,1558378898,4,24.01,-104.61,84.20,16.11
545,545,veraval,0,FR,1558378898,62,49.65,0.71,61.00,8.05
546,546,novyy urgal,100,RU,1558378899,93,51.07,132.56,49.74,2.68


In [4]:
locations = city_df_complete[["Lat", "Lng"]].astype(float)

In [5]:
# Create a Humidity Heatmap layer
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations , weights=city_df_complete.Humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
new_df = city_df_complete[city_df_complete["Max Temp"] < 70]
new_df = new_df[new_df["Max Temp"] > 60]
new_df = new_df[new_df["Wind Speed"] < 5]
new_df = new_df[new_df["Cloudiness"]!= 0]
new_df = new_df[new_df["Cloudiness"]!= 100]
new_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70
86,86,nikolskoye,20,RU,1558378770,56,59.70,30.79,69.01,4.47
107,107,whitehorse,40,CA,1558378774,36,60.72,-135.06,64.00,3.36
135,135,kibaya,75,RW,1558378779,88,-2.20,30.56,66.20,3.36
149,149,ganzhou,1,CN,1558378783,66,25.86,114.93,64.32,3.42
186,186,yamada,75,JP,1558378789,93,36.58,137.08,63.00,2.24
215,215,le port,25,FR,1558378795,64,42.87,1.37,60.01,4.47
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42
238,238,bose,75,IT,1558378816,72,45.46,8.01,64.40,2.24
249,249,lebu,75,ET,1558378826,68,8.96,38.73,64.40,2.95


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [7]:
new_df["Hotel Name"] = ''
new_df

,City_ID,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
69,69,fortuna,20,ES,1558378767,77,38.18,-1.13,69.80,4.70,
86,86,nikolskoye,20,RU,1558378770,56,59.70,30.79,69.01,4.47,
107,107,whitehorse,40,CA,1558378774,36,60.72,-135.06,64.00,3.36,
135,135,kibaya,75,RW,1558378779,88,-2.20,30.56,66.20,3.36,
149,149,ganzhou,1,CN,1558378783,66,25.86,114.93,64.32,3.42,
186,186,yamada,75,JP,1558378789,93,36.58,137.08,63.00,2.24,
215,215,le port,25,FR,1558378795,64,42.87,1.37,60.01,4.47,
231,231,mocuba,1,MZ,1558378804,90,-16.84,36.99,65.76,2.42,
238,238,bose,75,IT,1558378816,72,45.46,8.01,64.40,2.24,
249,249,lebu,75,ET,1558378826,68,8.96,38.73,64.40,2.95,


In [16]:
for city in new_df.iterrows():
    try:
        lat = city[1]['Lat']
        long = city[1]['Lng']
        # geocoordinates
        target_coordinates = f"{lat} , {long}"
        target_radius = 5000
        target_type = "hotel"
        # set up a parameters dictionary
        params = {
            "location": target_coordinates,
            "radius": target_radius,
            "type": target_type,
            "key": g_key
        }
        print(target_coordinates)
        # base url
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

        # run a request using our params dictionary
        response = requests.get(base_url, params=params)

        # convert response to json
        places_data = response.json()
        hotel_nm = places_data['results'][1]['name']
        print(hotel_nm)
        #add into df
        new_df['Hotel Name'].loc[city] == hotel_nm
    except:
        print('except')


38.18 , -1.13
Balneario de Leana
except
59.7 , 30.79
Nikolskaya College № 1
except
60.72 , -135.06
Westmark Hotel
except
-2.2 , 30.56
KCB Iwacu
except
25.86 , 114.93
Yangmeidu Bridge
except
36.58 , 137.08
Soba noodle shop
except
42.87 , 1.37
Chevrel Loïc
except
-16.84 , 36.99
Phama Lodge
except
45.46 , 8.01
Taverna Verde
except
8.96 , 38.73
Dah-Lak Guest House
except
-0.82 , -78.63
AGROGANA - Agroganadera Espinosa Chiriboga S.A. La Esperanza
except
-0.91 , 122.88
ATM BNI SPBU BIAK
except
38.73 , 35.48
ibis Kayseri
except
45.88 , 10.89
Hotel Garda
except
-15.0 , 22.67
Nalionwa Day Secondary School
except
29.03 , 66.58
MCH Center
except
-28.65 , 153.62
Backpackers Inn on the Beach
except
43.56 , 6.97
Hôtel Château de la Tour
except
-18.65 , 146.16
Lees Hotel
except
30.51 , 117.04
Alliance
except
52.06 , -2.72
The Green Dragon Hotel
except
36.62 , -121.92
Lovers Point Inn
except
6.99 , 35.58
Bench Maji Coffee Union HQ
except
-10.88 , 26.6
Societe Gecamines Kambove
except
61.99 , 5.14
Nord

In [19]:
hotel_df = new_df

In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [23]:
# Add marker layer ontop of heat map
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))